In [5]:
!pip install spacy
!pip install fake_useragent
! python -m spacy download ru_core_news_sm

2023-10-26 07:31:30.305082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 07:31:30.305139: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 07:31:30.305163: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 07:31:30.310549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 07:31:31.234145: W tensorflow/compiler/

In [6]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
import time

In [7]:
session = requests.session()
ua = UserAgent()

In [8]:
texts = []
authors = []
titles = []
#это списки, в которых будут храниться метаданные по всем текстам

In [9]:
urls = []
sets_of_stories = [
    'https://ficbook.net/readfic/10332769',
    'https://ficbook.net/readfic/12931291',
    'https://ficbook.net/readfic/13710114',
    'https://ficbook.net/readfic/018b3c1a-913f-7e4f-931e-37d0997ad3c6',
    'https://ficbook.net/readfic/9286325',
    'https://ficbook.net/readfic/018b1a2f-1bbc-7205-b029-d78a18ed79e4',
    'https://ficbook.net/readfic/0189933d-b420-7d6e-bb33-e92fb5d567fd',
    'https://ficbook.net/readfic/13550055',
    'https://ficbook.net/readfic/13030645',
    'https://ficbook.net/readfic/4720276'
]
#тут странички с ссылками на сборники рассказов, из них мы достаем много ссылок на конкретные рассказы и пишем в urls

In [10]:
def parser(url):
    headers = {'User-Agent': ua.random}
    response = session.get(url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    texts.append(soup.find('div', {'itemprop': 'articleBody'}).text)
    authors.append(soup.find('a', {'itemprop': 'author'}).text)
    titles.append(soup.find('h2', {'itemprop': 'headline'}).text)
#тут функция по доставанию всех нужных метадынных со страницы

In [11]:
def getting_urls(url):
    headers = {'User-Agent': ua.random}
    response = session.get(url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    links = soup.find_all('a', {'class': 'part-link visit-link'})
    for link in links:
        urls.append('https://ficbook.net' + link.get('href'))
#тут функция как из страницы со сборником достать ссылки на конкретные рассказы

In [12]:
for link in sets_of_stories:
  getting_urls(link)

In [13]:
for url in urls:
  parser(url)
  time.sleep(0.3)
#там блокируется доступ к страничке, поэтому делаем паузы между запросами

In [14]:
import re
for i, text in enumerate(texts):
  texts[i] = re.sub(r'[\t\n\r\xa0]', '', text)
  texts[i] = re.sub(r'([.,!?:;])', r'\1 ', texts[i])
  texts[i] = re.sub(r' *([.,!?:;]) *', r'\1 ', texts[i])
  texts[i] = texts[i].replace(' .', '.')
  texts[i] = texts[i].replace(' !', '!')
  texts[i] = texts[i].replace(' ?', '?')
  texts[i] = texts[i].replace('___________________________________________', '')
#вырезаем все лишние табы, расставляем пробелы после знаков препинания, где не стоят,
#исправляем многоточие и несколько восклицательных/вопросительных знаков подряд,
#удаляем длинную полоску из нижних подчеркиваний, которой разделяют части одного текста

In [15]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
sentences = []
#тут будут записаны все предложения, их же мы будем выводить в результате поиска
ids = []
#тут каждому предложению ставится в соответствие id текста, из которого оно берется, чтобы потом выводить вместе с предложением все метаданные про текст
#это id = индекс для каждого списка с метаданными
for i, text in enumerate(texts):
  sents = sent_tokenize(text)
  for sent in sents:
    sentences.append(sent)
    ids.append(i)

In [17]:
import spacy

nlp = spacy.load("ru_core_news_sm")

lemmas = []
pos_tags = []
word_forms = []
#это списки списков, где каждый список соответсвует набору pos тэгов/лемм/словоформ из конкретного предложения (такие же индексы как в списке предложений,
#так что lemmas[i] - это список лемм из предложения sentences[i]), поэтому если надо искать леммы, то мы ищем по списку лемм, и, если нашли, выводим предложение
#из списка предложений со всей метаинформацией


for sent in sentences:
  list_of_pos = []
  list_of_lemmas = []
  list_of_word_forms = []
  doc = nlp(sent)
  for token in doc:
    list_of_pos.append(token.pos_)
    list_of_lemmas.append(token.lemma_)
    list_of_word_forms.append(token.text)
  lemmas.append(list_of_lemmas)
  pos_tags.append(list_of_pos)
  word_forms.append(list_of_word_forms)

# Новый раздел

In [18]:
print(sentences[69])
print(lemmas[69])
print(pos_tags[69])
print(word_forms[69])
print(ids[69])
print(authors[ids[69]])
print(titles[ids[69]])
print(urls[ids[69]])
#тут примеры, что выводится в каждом списке и как они связаны

Они застыли на долгие минуты, приходя в себя, восстанавливая дыхание и обращаясь к Силе за утешением.
['они', 'застынуть', 'на', 'долгий', 'минута', ',', 'приходить', 'в', 'себя', ',', 'восстанавливать', 'дыхание', 'и', 'обращаться', 'к', 'сила', 'за', 'утешение', '.']
['PRON', 'VERB', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'ADP', 'PRON', 'PUNCT', 'VERB', 'NOUN', 'CCONJ', 'VERB', 'ADP', 'PROPN', 'ADP', 'NOUN', 'PUNCT']
['Они', 'застыли', 'на', 'долгие', 'минуты', ',', 'приходя', 'в', 'себя', ',', 'восстанавливая', 'дыхание', 'и', 'обращаясь', 'к', 'Силе', 'за', 'утешением', '.']
0
stratching in the sky
Джедай и падаван
https://ficbook.net/readfic/10332769/35720468#part_content


In [19]:
#стандартный тегсет UD
pos_list = ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']

In [116]:
def search(query):
    tokens = query.split()
    results = []

    if len(tokens) > 3:
      print('в запросе должно быть не больше 3 токенов')

    else:
      for idx, sent in enumerate(sentences):
          token_pointer = 0  # Указатель для токенов запроса
          element_pointer = 0  # Указатель для элементов предложения (лемм или POS-тегов или словоформ)

          while token_pointer < len(tokens) and element_pointer < len(lemmas[idx]):
              current_token = tokens[token_pointer]

              #для одного токена, чтобы не проверять на последовательное расположение токенов в тексте
              if len(tokens) == 1:

                  # Для словоформы в кавычках
                  if current_token.startswith('"') and current_token.endswith('"'):
                    word = current_token[1:-1]
                    if word == word_forms[idx][element_pointer]:
                        token_pointer += 1
                    element_pointer += 1 # Move to next element regardless of the current token match

                  # Для комбинации леммы и POS-тега
                  elif '+' in current_token:
                      word, pos = current_token.split('+')
                      if word == lemmas[idx][element_pointer] and pos == pos_tags[idx][element_pointer]:
                          token_pointer += 1
                          element_pointer += 1
                          continue
                      else:
                          element_pointer += 1
                          continue

                  # Для поиска по POS-тегу
                  elif current_token in pos_list:
                      if current_token == pos_tags[idx][element_pointer]:
                          token_pointer += 1
                      element_pointer += 1 # Move to next element regardless of the current token match

                  # Для поиска по лемме/словоформе
                  else:
                    if current_token == lemmas[idx][element_pointer] or current_token == word_forms[idx][element_pointer]:
                        token_pointer += 1
                    element_pointer += 1 # Move to next element regardless of the current token match

              else:

                # Для словоформы в кавычках
                if current_token.startswith('"') and current_token.endswith('"'):
                    word = current_token[1:-1]
                    if word == word_forms[idx][element_pointer]:
                        token_pointer += 1
                        element_pointer += 1
                    else:
                      break

                # Для комбинации леммы и POS-тега
                elif '+' in current_token:
                    word, pos = current_token.split('+')
                    if word == lemmas[idx][element_pointer] and pos == pos_tags[idx][element_pointer]:
                        token_pointer += 1
                        element_pointer += 1
                        continue
                    else:
                        element_pointer += 1
                        continue

                # Для поиска по POS-тегу
                elif current_token in pos_list:
                    if current_token == pos_tags[idx][element_pointer]:
                        token_pointer += 1
                        element_pointer += 1
                    else:
                        break

                # Для поиска по лемме/словоформе
                else:
                    if current_token == lemmas[idx][element_pointer] or current_token == word_forms[idx][element_pointer]:
                        token_pointer += 1
                        element_pointer += 1
                    else:
                      break

          # Если все токены запроса были найдены в предложении в нужной последовательности
          if token_pointer == len(tokens):
              meta_info = f"{titles[ids[idx]]} - {authors[ids[idx]]} ({urls[ids[idx]]})"
              results.append((sent, meta_info))

      for res, meta in results:
          print(res)
          print(f"Source: {meta}\n")
      if results == []:
        print('Ничего не найдено')


In [109]:
search('знать+VERB ситх')

Ничего не найдено


In [101]:
search('NOUN VERB NOUN VERB')

в запросе должно быть не больше 3 токенов


In [110]:
search('знать+VERB NOUN')

Кому еще знать анатомию, как не убийце.
Source: Незваная гостья - Jarethina (https://ficbook.net/readfic/9286325/23826185#part_content)

—Ты будешь знать правду.
Source: Маленький инцидент - Jarethina (https://ficbook.net/readfic/9286325/23873351#part_content)

Она удовольствовалась учебниками по перспективе и композиции, а еще скупыми объяснениями самого Кайло, который знал основы академического рисунка, и видеоуроками.
Source: Маленький инцидент - Jarethina (https://ficbook.net/readfic/9286325/23873351#part_content)

Но при этом ты очень хорошо держал оборону, будто уже зная соресу.
Source: Турнир - Постмодерн (https://ficbook.net/readfic/0189933d-b420-7d6e-bb33-e92fb5d567fd/35219117#part_content)

От падавана шла темнота и на самом деле он уже знал ответ на свой вопрос.
Source: Будни падавана - Постмодерн (https://ficbook.net/readfic/0189933d-b420-7d6e-bb33-e92fb5d567fd/35269449#part_content)

Он прекрасно знал ответ на этот вопрос, но не хотел на него отвечать.
Source: Я и Мой пада

In [111]:
search('NOUN ситх')

Магия ситхов?
Source: Мастер, тогрута и тогрута - stratching in the sky (https://ficbook.net/readfic/10332769/35726040#part_content)

Тирания ситхов расцвела в галактике за минувшие пятнадцать лет буйным цветом, отравив всё вокруг корнями Темной Стороны.
Source: Путь к кооперации - Krosskun (https://ficbook.net/readfic/4720276/25263643#part_content)



In [112]:
search('VERB VERB VERB')

Желающих подойти поговорить с ней было немного, а те, кто подходил, хотели попасть в команду с «непобедимой девочкой», а не с Пиррой Никос.
Source: JNPR NG+ (alt RWBY) - H-eon (https://ficbook.net/readfic/13030645/33653211#part_content)

Стараясь использовать начавшийся пожар как дымовую завесу, он пополз прочь от линии соприкосновения, держа винтовку в одной руке, а термальный детонатор в другой, как последний довод, который он мог применить в свою защиту.
Source: Сопротивление обречённых - Krosskun (https://ficbook.net/readfic/4720276/12249473#part_content)

Начал нараспев читать свои предсказания высокий прорицатель.
Source: Эхо грядущего - Krosskun (https://ficbook.net/readfic/4720276/12251120#part_content)



In [115]:
search('"ситха"')

Я же не настолько глупа, чтобы ходить с именем, под которым меня знали, как ситха, вот и сменила!
Source: О том, почему у Асоки фамилия «Непей-Пиво» и причём тут Асажж - Ёкарная Бабайка (https://ficbook.net/readfic/12931291/33486905#part_content)

— возразил Леший, несколько раз стреляя в ситха.
Source: Осквернённый монастырь - Мистер Вульф (https://ficbook.net/readfic/13710114/35127666#part_content)

Перед ним предстала голограмма ситха.
Source: Будни падавана - Постмодерн (https://ficbook.net/readfic/0189933d-b420-7d6e-bb33-e92fb5d567fd/35269449#part_content)

— осторожно спросил офицер связи у адмирала, но тот не ответил, и только умоляюще посмотрел на устрашающую фигуру владыки-ситха, чьё тяжёлое механическое дыхание единственное нарушало царившую вокруг тишину.
Source: Новые тактические приёмы - Krosskun (https://ficbook.net/readfic/4720276/12234981#part_content)

— чёрная маска скрывала лицо ситха, но, даже искажённый дыхательной системой, его голос не скрывал явного удивления.
S

In [117]:
search('ситх')

— Только ситхи мыслят о вечной жизни и воскрешении.
Source: Мастер, тогрута и тогрута - stratching in the sky (https://ficbook.net/readfic/10332769/35726040#part_content)

— Энакин ситх?
Source: Мастер, тогрута и тогрута - stratching in the sky (https://ficbook.net/readfic/10332769/35726040#part_content)

Магия ситхов?
Source: Мастер, тогрута и тогрута - stratching in the sky (https://ficbook.net/readfic/10332769/35726040#part_content)

— А ты…— Я это я. Не клон, подделка ситхов или еще что-то, что ты можешь напридумывать.
Source: Мастер, тогрута и тогрута - stratching in the sky (https://ficbook.net/readfic/10332769/35726040#part_content)

Об этом кричала Сила, яркая от присутствия стольких джедаев, не окровавленная миллиардами жертв и все еще не посрамленная зверствами ситхов.
Source: Добро пожаловать домой - stratching in the sky (https://ficbook.net/readfic/10332769/35761481#part_content)

Смерть падавана Тано была Ситху на руку, открывала для него новые перспективы и позволяла уск

In [118]:
search('знать+NOUN')

— Хм, эти мундиры и оружие весьма древние, я думаю, они относятся к временам старой знати.
Source: Бессмысленный меч (Rurouni Kenshin (фильм)) - H-eon (https://ficbook.net/readfic/13030645/33589158#part_content)

